In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

# Tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

# Some libraries
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns

input_file = np.load('/content/drive/My Drive/public_data.npz', allow_pickle=True)
data = input_file['data']

labels = input_file['labels']

label_dict = {'healthy': 0, 'unhealthy': 1}
labels = np.array([label_dict[label] for label in labels])


2.14.0


In [36]:
#data = data/255.0

from keras.applications.efficientnet_v2 import preprocess_input
data = preprocess_input(data)

In [37]:
shrek_indices = []
trol_indices = []
new_data = []
new_labels = []
for i, image in enumerate(data):
  if np.sum(data[506] - image) == 0:
    shrek_indices.append(i)
  elif np.sum(data[338] - image) == 0:
    trol_indices.append(i)
  else:
    new_data.append(image)
    new_labels.append(labels[i])

data = np.array(new_data)
labels = np.array(new_labels)

labels = tfk.utils.to_categorical(labels)

In [38]:
X_train_val, X_test, y_train_val, y_test = train_test_split(data, labels, test_size=0.2, random_state=seed)

In [39]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=seed)

In [40]:
input_shape = X_train.shape[1:]  # Input shape for the model
output_shape = 2  # Output shape for the model

In [41]:
learned_model = tfk.applications.EfficientNetV2S(
    input_shape=(96, 96, 3),
    include_top=False,
    weights="imagenet",
    pooling='max',
)

In [42]:
for i, layer in enumerate(learned_model.layers):
   print(i, layer.name, layer.trainable)

0 input_3 True
1 rescaling_1 True
2 stem_conv True
3 stem_bn True
4 stem_activation True
5 block1a_project_conv True
6 block1a_project_bn True
7 block1a_project_activation True
8 block1a_add True
9 block1b_project_conv True
10 block1b_project_bn True
11 block1b_project_activation True
12 block1b_drop True
13 block1b_add True
14 block2a_expand_conv True
15 block2a_expand_bn True
16 block2a_expand_activation True
17 block2a_project_conv True
18 block2a_project_bn True
19 block2b_expand_conv True
20 block2b_expand_bn True
21 block2b_expand_activation True
22 block2b_project_conv True
23 block2b_project_bn True
24 block2b_drop True
25 block2b_add True
26 block2c_expand_conv True
27 block2c_expand_bn True
28 block2c_expand_activation True
29 block2c_project_conv True
30 block2c_project_bn True
31 block2c_drop True
32 block2c_add True
33 block2d_expand_conv True
34 block2d_expand_bn True
35 block2d_expand_activation True
36 block2d_project_conv True
37 block2d_project_bn True
38 block2d_drop

In [43]:
learned_model.trainable = True

In [44]:
# Freeze first N layers, e.g., until the 133rd one
N = 105
for i, layer in enumerate(learned_model.layers[:N]):
  layer.trainable=False
for i, layer in enumerate(learned_model.layers):
   print(i, layer.name, layer.trainable)


0 input_3 False
1 rescaling_1 False
2 stem_conv False
3 stem_bn False
4 stem_activation False
5 block1a_project_conv False
6 block1a_project_bn False
7 block1a_project_activation False
8 block1a_add False
9 block1b_project_conv False
10 block1b_project_bn False
11 block1b_project_activation False
12 block1b_drop False
13 block1b_add False
14 block2a_expand_conv False
15 block2a_expand_bn False
16 block2a_expand_activation False
17 block2a_project_conv False
18 block2a_project_bn False
19 block2b_expand_conv False
20 block2b_expand_bn False
21 block2b_expand_activation False
22 block2b_project_conv False
23 block2b_project_bn False
24 block2b_drop False
25 block2b_add False
26 block2c_expand_conv False
27 block2c_expand_bn False
28 block2c_expand_activation False
29 block2c_project_conv False
30 block2c_project_bn False
31 block2c_drop False
32 block2c_add False
33 block2d_expand_conv False
34 block2d_expand_bn False
35 block2d_expand_activation False
36 block2d_project_conv False
37 bl

In [45]:
preprocessing = tf.keras.Sequential([
    tfkl.RandomFlip("horizontal_and_vertical"),
    tfkl.RandomRotation(0.5),
    #tfkl.RandomTranslation(0.2,0.2),
    #tfkl.RandomContrast(0.2),
    #tfkl.RandomRotation(0.2)
], name='preprocessing')


input_layer = tfkl.Input(shape=input_shape)
x = preprocessing(input_layer)

In [46]:
x = learned_model(x)

In [47]:
print(x.shape)

(None, 1280)


In [48]:
x = tfkl.Flatten(name='flattenLast')(x)
x = tfkl.Dense(1028, activation='relu', name='dense_OLast')(x)
x = tfkl.Dense(512, activation='relu', name='dense_1Last')(x)
x = tfkl.Dense(64, activation='relu', name='dense_2Last')(x)
x = tfkl.Dense(16, activation='relu', name='dense_3Last')(x)
output_layer = tfkl.Dense(output_shape, activation='sigmoid', name='output')(x)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

In [49]:
model.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(1e-5), metrics='accuracy')
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 preprocessing (Sequential)  (None, 96, 96, 3)         0         
                                                                 
 efficientnetv2-s (Function  (None, 1280)              20331360  
 al)                                                             
                                                                 
 flattenLast (Flatten)       (None, 1280)              0         
                                                                 
 dense_OLast (Dense)         (None, 1028)              1316868   
                                                                 
 dense_1Last (Dense)         (None, 512)               526848    
                                                           

In [50]:
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)

In [51]:
model_checkpoint = tfk.callbacks.ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')

In [52]:
from sklearn.utils.class_weight import compute_class_weight
#class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
#class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights_dict = {0: 0.81, 1: 1.29}

print(f"Class weights: {class_weights_dict}")

Class weights: {0: 0.81, 1: 1.29}


In [53]:
batch_size=16
epochs=45
history = model.fit(
    x=data,
    y=labels,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_val, y_val),
    class_weight=class_weights_dict,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

Epoch 1/45
313/313 [==============================] - 132s 168ms/step - loss: 0.6708 - accuracy: 0.6159 - val_loss: 0.6224 - val_accuracy: 0.6767
Epoch 2/45
313/313 [==============================] - 45s 144ms/step - loss: 0.5897 - accuracy: 0.7080 - val_loss: 0.5377 - val_accuracy: 0.7341
Epoch 3/45
313/313 [==============================] - 45s 143ms/step - loss: 0.5368 - accuracy: 0.7398 - val_loss: 0.4885 - val_accuracy: 0.7640
Epoch 4/45
313/313 [==============================] - 49s 158ms/step - loss: 0.5118 - accuracy: 0.7560 - val_loss: 0.4675 - val_accuracy: 0.7828
Epoch 5/45
313/313 [==============================] - 42s 134ms/step - loss: 0.4832 - accuracy: 0.7658 - val_loss: 0.4360 - val_accuracy: 0.7940
Epoch 6/45
313/313 [==============================] - 47s 149ms/step - loss: 0.4621 - accuracy: 0.7790 - val_loss: 0.4056 - val_accuracy: 0.8152
Epoch 7/45
313/313 [==============================] - 44s 139ms/step - loss: 0.4609 - accuracy: 0.7766 - val_loss: 0.4087 - val_a

In [54]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

32/32 [==============================] - 1s 42ms/step - loss: 0.1493 - accuracy: 0.9471
Test loss: 0.14932982623577118
Test accuracy: 0.9470529556274414


In [55]:
A, X_test_new, b, y_test_new = train_test_split(X_test, y_test, test_size=0.1)
test_loss, test_accuracy = model.evaluate(X_test_new, y_test_new, verbose=1)
print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

4/4 [==============================] - 0s 44ms/step - loss: 0.1930 - accuracy: 0.9109
Test loss: 0.1929883509874344
Test accuracy: 0.9108911156654358


In [56]:
model.save('SumbissionModelXceptionLastLast')